# Streaming Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.


In [ ]:
!python -m pip install semantic-kernel==1.0.5

In [2]:
import sys
import os

# Add the samples directory to the Python path
samples_dir = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples'
sys.path.append(samples_dir)

try:
    from services import Service
    from service_settings import ServiceSettings

    service_settings = ServiceSettings(
        global_llm_service="AzureOpenAI")  # Example setting

    # Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
    selectedService = (
        Service.AzureOpenAI
        if service_settings.global_llm_service is None
        else Service(service_settings.global_llm_service.lower())
    )
    print(f"Using service type: {selectedService}")
except ModuleNotFoundError as e:
    print(f"Error: {e}")
    print("Ensure that 'service_settings.py' is in the correct directory and properly named.")
    # Additional debugging information
    print("Current Python path:")
    for path in sys.path:
        print(path)

Using service type: Service.AzureOpenAI


First, we will set up the text and chat services we will be submitting prompts to.


In [3]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.hugging_face import (  # noqa: F401
    HuggingFacePromptExecutionSettings,
    HuggingFaceTextCompletion,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings  # noqa: F401
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings  # noqa: F401
from semantic_kernel.connectors.ai.open_ai import OpenAITextPromptExecutionSettings  # noqa: F401
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
    OpenAIChatCompletion,
    OpenAITextCompletion,
)
from dotenv import load_dotenv

env_path = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples\getting_started\.env'
load_dotenv(env_path)

service_id = None

# Load the API keys and other settings from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_org_id = os.getenv('OPENAI_ORG_ID')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_chat_deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

kernel = Kernel()

# Configure Azure LLM service
if selectedService == Service.AzureOpenAI:
    azure_text_service = AzureTextCompletion(
        service_id="aoai_text",
        api_key=azure_openai_api_key,           # Use the api_key from the .env file
        # Use the deployment_name from the .env file
        deployment_name=azure_chat_deployment_name,
        endpoint=azure_openai_endpoint,         # Use the endpoint from the .env file
        env_file_path=env_path,
    )  # set the deployment name to the value of your text model (e.g. gpt-35-turbo-instruct)
    azure_chat_service = AzureChatCompletion(
        service_id="aoai_chat",
        api_key=azure_openai_api_key,           # Use the api_key from the .env file
        # Use the deployment_name from the .env file
        deployment_name=azure_chat_deployment_name,
        endpoint=azure_openai_endpoint,         # Use the endpoint from the .env file
        env_file_path=env_path,
    )  # set the deployment name to the value of your chat model



# Configure Hugging Face service


if selectedService == Service.HuggingFace:


    hf_text_service = HuggingFaceTextCompletion(ai_model_id="distilgpt2", task="text-generation")

c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Next, we'll set up the completion request settings for text completion services.


In [4]:
oai_prompt_execution_settings = OpenAITextPromptExecutionSettings(
    service_id="oai_text",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

## Streaming Open AI Text Completion


In [5]:
if selectedService == Service.OpenAI:
    prompt = "What is the purpose of a rubber duck?"
    stream = oai_text_service.get_streaming_text_contents(prompt=prompt, settings=oai_prompt_execution_settings)
    async for message in stream:
        print(str(message[0]), end="")  # end = "" to avoid newlines

## Streaming Azure Open AI Text Completion


In [6]:
if selectedService == Service.AzureOpenAI:
    prompt = "provide me a list of possible meanings for the acronym 'ORLD'"
    stream = azure_text_service.get_streaming_text_contents(prompt=prompt, settings=oai_prompt_execution_settings)
    async for message in stream:
        print(str(message[0]), end="")

 "Open Research and Learning Database" or "Online Resources for Learning and Development"

For the ORLD acronym, there are a few possible meanings: 

- Open Research and Learning Database 
- Online Resources for Learning and Development 
- Organization of Rural Land Developers 
- Oil Refining and Logistics Division 
- Organic Recycling and Landscaping Design 
- Operations Research and Logistics Division 
- Office of Real Estate Licensure and Development 
- Olympic Rowing Legacy Development 

Note that some of these are less likely than others, but they are all possible interpretations based on the letters in the acronym. The most common meanings for ORLD are probably Open Research and Learning Database or Online Resources for Learning and Development.

## Streaming Hugging Face Text Completion


In [7]:
if selectedService == Service.HuggingFace:
    hf_prompt_execution_settings = HuggingFacePromptExecutionSettings(
        service_id="hf_text",
        extension_data={
            "max_new_tokens": 80,
            "top_p": 1,
            "eos_token_id": 11,
            "pad_token_id": 0,
        },
    )

In [8]:
if selectedService == Service.HuggingFace:
    prompt = "The purpose of a rubber duck is"
    stream = hf_text_service.get_streaming_text_contents(
        prompt=prompt, prompt_execution_settings=hf_prompt_execution_settings
    )
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines

Here, we're setting up the settings for Chat completions.


In [9]:
oai_chat_prompt_execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="oai_chat",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

## Streaming OpenAI Chat Completion


In [10]:
if selectedService == Service.OpenAI:
    content = "You are an AI assistant that helps people find information."
    chat = ChatHistory()
    chat.add_system_message(content)
    stream = oai_chat_service.get_streaming_chat_message_contents(
        chat_history=chat, settings=oai_chat_prompt_execution_settings
    )
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines

## Streaming Azure OpenAI Chat Completion


In [11]:
az_oai_chat_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service_id="aoai_chat",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

In [12]:
if selectedService == Service.AzureOpenAI:
    content = "You are an AI assistant that helps people find information."
    chat = ChatHistory()
    chat.add_system_message(content)
    chat.add_user_message("What is the purpose of a rubber duck?")
    stream = azure_chat_service.get_streaming_chat_message_contents(
        chat_history=chat, settings=az_oai_chat_prompt_execution_settings
    )
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines

NameError: name 'ChatHistory' is not defined